In [1]:
########### IMPORTING THE REQURIED LIBRARIES ###########

from bs4 import BeautifulSoup as soup
import requests
import json
from random import choice
from terminaltables import AsciiTable

In [2]:
BASE_URL = "https://www.worldometers.info/coronavirus/#countries"
MAX_TIMEOUT = 3

In [3]:
######## DECLARING THE CLASS FOR SCRAPING SSLPROXIES.COM ########

class _proxy:

	PROXY_URL = 'https://sslproxies.org/'
	page = None

	def __init__( self ):
		self.page = soup( requests.get( self.PROXY_URL ).text, 'lxml' )
    
	def getSSLProxyDictionary( self ):
 
		return {
		'https': choice( list( map( lambda x:x[ 0 ] + ':' + x[ 1 ], list( zip( map( lambda x:x.text, self.page.find( 'table' ).findAll( 'td' )[ ::8 ] ), map( lambda x:x.text, self.page.find( 'table' ).findAll( 'td' )[ 1::8 ] ) ) ) ) ) )
		}

In [4]:
proxy = _proxy()

def _loadDataByIPRotaion( url ):
	count = 0
	response = None
    
	while count < 10:
		try:
			proxyDictionary = proxy.getSSLProxyDictionary()

			print( 'Retry {} Using Proxy : {}'.format( count, proxyDictionary ) )

			response = requests.get( url, proxies = proxyDictionary, timeout = MAX_TIMEOUT )
			break
		except:
			pass

		count=count+1

	return response

In [5]:
try:
    resp = requests.get( BASE_URL, timeout = MAX_TIMEOUT )
    page = soup( resp.text, 'lxml' ) 
except:
    resp = _loadDataByIPRotaion( BASE_URL )
    page = soup( resp.text, 'lxml' )

In [38]:
table = page.find( "table",{
   "id": "main_table_countries_today" 
} )

In [39]:
table_heading = [ item.text.strip() for item in table.thead.tr if item != "\n" ]

In [40]:
table_content = []
for rows in table.tbody:
    data = [ item.text.strip() for item in rows if item != "\n" ]
    if data:
        table_content.append( data[ : -2 ] )

In [41]:
len(table_content)

197

In [42]:
table_content.insert( 0, table_heading[ : -2 ] )

In [43]:
table = AsciiTable(table_content)
print(table.table)

+------------------------+------------+----------+-------------+-----------+----------------+-------------+------------------+
| Country,Other          | TotalCases | NewCases | TotalDeaths | NewDeaths | TotalRecovered | ActiveCases | Serious,Critical |
+------------------------+------------+----------+-------------+-----------+----------------+-------------+------------------+
| China                  | 81,171     | +78      | 3,277       | +7        | 73,159         | 4,735       | 1,573            |
| Italy                  | 69,176     | +5,249   | 6,820       | +743      | 8,326          | 54,030      | 3,393            |
| USA                    | 49,594     | +5,860   | 622         | +69       | 361            | 48,611      | 1,175            |
| Spain                  | 39,676     | +4,540   | 2,800       | +489      | 3,794          | 33,082      | 2,355            |
| Germany                | 31,991     | +2,935   | 149         | +26       | 749            | 31,093      | 23 

In [12]:
table_heading

['Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Serious,Critical',
 'Tot\xa0Cases/1M pop',
 'Tot\xa0Deaths/1M pop']